In [4]:
%%time
import pandas as pd

from sklearn.metrics import roc_auc_score, make_scorer, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from sklearn.feature_selection import RFE


from time import strftime, strptime, mktime, time, ctime

from sklearn import model_selection

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier,\
ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier

from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from mlxtend.feature_selection import ColumnSelector

from sklearn.pipeline import make_pipeline
import numpy as np

from sklearn.feature_selection import RFE
import gc


/home/shj16110/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CPU times: user 516 ms, sys: 1.85 s, total: 2.37 s
Wall time: 9.57 s


In [2]:
%%time
df = pd.read_csv('./train.csv')
print df.shape
test = pd.read_csv('./test.csv')
print test.shape

y = df.pop('Approved')
train_id = df.pop('ID')
test_id = test.pop('ID')

def helper(x):
    try:
        tmp = strptime(x, '%d/%m/%y')
        tmp = mktime(tmp)
    except:
        tmp = 563000400
    return tmp

df['Lead_Creation_Date'] = df['Lead_Creation_Date'].apply(helper)
test['Lead_Creation_Date'] = test['Lead_Creation_Date'].apply(helper)

df['DOB'] = df['DOB'].apply(helper)
test['DOB'] = test['DOB'].apply(helper)

df = df.fillna(axis=1, value=-1)
test = test.fillna(axis=1, value=-1)

df = pd.get_dummies(df)
test = pd.get_dummies(test)
test = test.reindex(columns = df.columns, fill_value=0)

s = StandardScaler()
df = s.fit_transform(df)
test = s.transform(test)

print df.shape, test.shape

(69713, 22)
(30037, 21)
(69713, 37416) (30037, 37416)
CPU times: user 55.4 s, sys: 36.3 s, total: 1min 31s
Wall time: 1min 31s


In [7]:
# np.save('test_id.npy', np.array(test_id))

In [13]:
!ls -sh|grep yes

8.4G yesterday_test.npy
 20G yesterday_train.npy


In [2]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G       6.2G       119G       168K       180M       772M
-/+ buffers/cache:       5.3G       120G 
Swap:         4.0G         0B       4.0G 


### from here

In [5]:
%%time
train = np.load('yesterday_train.npy')
test = np.load('yesterday_test.npy')
y = np.load('y.npy')
test_id = np.load('test_id.npy')

CPU times: user 11 ms, sys: 9 s, total: 9.01 s
Wall time: 25.9 s


In [6]:
!free -h

             total       used       free     shared    buffers     cached
Mem:          125G        34G        91G       168K       180M       772M
-/+ buffers/cache:        33G        92G 
Swap:         4.0G         0B       4.0G 


In [18]:
RANDOM_STATE = 2018
params = {
    'min_child_weight': 0.8,
#     'eta': 0.1,
    'colsample_bytree': 0.8,
    'max_depth': 12,
    'subsample': 0.8,
#     'alpha': 0.3,
#     'lambda':0.9,
#     'gamma': 0.3,
    'silent': 0,
    'learning_rate':0.05,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'objective':"binary:logistic",
#     'eval_metric':'error'
#     'num_class':1
#     'eval_metric':'auc',
#         'n_thread':-1,

    
}


In [19]:
%%time
model=XGBClassifier(**params)
rfe = RFE(model, train.shape[1]*0.9, verbose=1, step=.05)
rfe = rfe.fit(train, y)
# summarize the selection of the attributes
print(rfe.support_)
# print(rfe.ranking_)
index2=rfe.support_

TypeError: __init__() got an unexpected keyword argument 'verbose_eval'

In [ ]:
%%time
print train.shape, test.shape
train, test = train[:,index2], test[:, index2]
print train.shape, test.shape

In [ ]:
%%time
m = lambda *x: [i.shape for i in x]
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state  = 7)
print m(X_train, y_train, X_test, y_test)

In [ ]:

%%time
d_train = xgb.DMatrix(X_train, label=y_train)
d_test = xgb.DMatrix(X_test, label=y_test)

In [ ]:
start=time()
watchlist = [(d_train, 'train'), (d_test, 'test')]
clf = xgb.train(params, d_train, 10000, watchlist, early_stopping_rounds=10, 
                verbose_eval=1,  maximize=True,
               )
                


'!!! used %.2f s'%(time()-start)

In [13]:
%%time
dtest = xgb.DMatrix(test)

CPU times: user 14.9 s, sys: 7.02 s, total: 21.9 s
Wall time: 21.9 s


In [14]:
%%time
predb = clf.predict(dtest)


CPU times: user 4.3 s, sys: 1 ms, total: 4.3 s
Wall time: 268 ms


___

In [19]:
submit = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(predb, columns=['Approved'])], axis=1)
submit.shape

(30037, 2)

In [21]:
submit.to_csv('xgb1.csv',index=False)

In [22]:
clf.save_model('xgb_0.852586.model')

In [441]:
roc_auc_score?

In [442]:
submit2 = pd.concat([pd.DataFrame(test_id.values, columns=['ID']), pd.DataFrame(pred, columns=['Approved'])], axis=1)
submit2.shape

(30037, 2)

In [449]:
accuracy_score(y_test, [0]*y_test.shape[0])

0.9834325467976762

___

In [276]:
cross_val_score(RandomForestClassifier(n_estimators=50, n_jobs=24), df.iloc[:,:-1], df.iloc[:,-1], cv=5)

array([0.87807502, 1.        , 0.99992828, 1.        , 0.98759145])

In [76]:
fr = RandomForestClassifier(n_estimators=50)
fr

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [77]:
fr.fit(df.iloc[:,:-1], df.iloc[:,-1])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [ ]:
fr.predict_log_proba